In [69]:
import numpy as np
import pandas as pd
from statsmodels.tsa.stattools import coint
import plotly.graph_objects as go
from joblib import Parallel, delayed
from sklearn.linear_model import LinearRegression
import re

## Data Preprocessing

In [72]:
ret_df = pd.read_feather('data/nasdaq_etfs.feather') # contains daily percentage returns (NOTE: 0.747 means 0.747% not 74.7%)

symmap = pd.read_csv("data/symbol2name.csv")
sym2name = dict(zip(symmap["Symbol"], symmap["Security Name"]))
syms2drop = []
for sym in ret_df.columns:
    name = sym2name.get(sym, None)
    if name is None: continue
    if re.search(r"\b(bill)\b", name.lower()):
        # print(sym, "|", name)
        syms2drop.append(sym)
ret_df.drop(columns=syms2drop, inplace=True)

ret_df.head()

Ticker,AAXJ,ACWI,ACWX,AFK,AGQ,AIA,AIVI,AIVL,AOA,AOK,...,XRT,XSD,XSMO,XSVM,YANG,YCL,YCS,YINN,YXI,ZSL
Date,,,,,,,,,,,,,,,,,,,,,
2010-07-13,0.747621,1.562114,1.686754,1.853138,2.700842,0.854489,1.788925,1.268345,1.884208,0.435343,...,2.734448,2.155864,3.351985,3.351523,-2.632439,0.140403,-0.369203,2.300923,-0.936347,-2.774568
2010-07-14,-0.380087,0.372102,0.263322,0.034330,1.457285,0.051332,0.051002,-0.168632,-0.470754,0.000000,...,-0.452477,0.065970,0.154452,-0.603311,2.571722,0.385555,-0.476437,-2.663477,1.155257,-1.634958
2010-07-15,-0.763130,0.098904,0.157541,0.274559,-0.511807,-0.821128,1.603865,0.192969,-0.135120,0.108410,...,0.588216,-0.131862,-0.462618,0.379324,2.828656,1.815644,-1.968086,-2.827608,0.000000,0.302206
2010-07-16,-2.453279,-2.765485,-2.884077,-1.540060,-4.829074,-2.664938,-2.706147,-2.167090,-2.063619,-0.721728,...,-3.561905,-2.947607,-3.718039,-3.930446,5.470490,2.023320,-1.627784,-5.882389,3.612906,5.122025
2010-07-19,1.144909,0.634864,0.539948,-0.451860,-2.580644,1.302489,0.566597,1.058404,0.207255,0.072730,...,0.385863,2.606528,0.402229,0.393409,-2.874933,-0.336136,0.386099,3.158260,-0.420834,2.694180


In [73]:
growth = 1 + ret_df/100
price_df = growth.cumprod().shift(1, fill_value=1)
price_df.head()

Ticker,AAXJ,ACWI,ACWX,AFK,AGQ,AIA,AIVI,AIVL,AOA,AOK,...,XRT,XSD,XSMO,XSVM,YANG,YCL,YCS,YINN,YXI,ZSL
Date,,,,,,,,,,,,,,,,,,,,,
2010-07-13,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2010-07-14,1.007476,1.015621,1.016868,1.018531,1.027008,1.008545,1.017889,1.012683,1.018842,1.004353,...,1.027344,1.021559,1.033520,1.033515,0.973676,1.001404,0.996308,1.023009,0.990637,0.972254
2010-07-15,1.003647,1.019400,1.019545,1.018881,1.041975,1.009063,1.018408,1.010976,1.014046,1.004353,...,1.022696,1.022233,1.035116,1.027280,0.998716,1.005265,0.991561,0.995762,1.002081,0.956358
2010-07-16,0.995988,1.020409,1.021151,1.021678,1.036642,1.000777,1.034742,1.012927,1.012676,1.005442,...,1.028712,1.020885,1.030328,1.031177,1.026966,1.023517,0.972046,0.967605,1.002081,0.959249
2010-07-19,0.971553,0.992189,0.991701,1.005944,0.986582,0.974107,1.006741,0.990976,0.991778,0.998186,...,0.992070,0.990793,0.992020,0.990647,1.083146,1.044226,0.956224,0.910687,1.038285,1.008381


In [ ]:
log_price_df = np.log(price_df)
log_price_df.head()

Ticker,AAXJ,ACWI,ACWX,AFK,AGQ,AIA,AIVI,AIVL,AOA,AOK,...,XRT,XSD,XSMO,XSVM,YANG,YCL,YCS,YINN,YXI,ZSL
Date,,,,,,,,,,,,,,,,,,,,,
2010-07-13,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2010-07-14,0.007448,0.015500,0.016727,0.018362,0.026650,0.008509,0.017731,0.012604,0.018667,0.004344,...,0.026977,0.021330,0.032970,0.032966,-0.026677,0.001403,-0.003699,0.022749,-0.009408,-0.028138
2010-07-15,0.003640,0.019215,0.019357,0.018705,0.041118,0.009022,0.018241,0.010916,0.013948,0.004344,...,0.022442,0.021989,0.034514,0.026914,-0.001285,0.005251,-0.008475,-0.004247,0.002079,-0.044623
2010-07-16,-0.004020,0.020203,0.020931,0.021447,0.035987,0.000777,0.034152,0.012844,0.012596,0.005427,...,0.028307,0.020670,0.029877,0.030701,0.026609,0.023245,-0.028352,-0.032931,0.002079,-0.041605
2010-07-19,-0.028859,-0.007841,-0.008334,0.005926,-0.013509,-0.026234,0.006718,-0.009065,-0.008256,-0.001816,...,-0.007962,-0.009250,-0.008012,-0.009397,0.079870,0.043276,-0.044764,-0.093556,0.037570,0.008347


In [75]:
# We use first 40% of data for cointegration test
train_size = int(len(log_price_df) * 0.40)
log_price_train = log_price_df.iloc[:train_size]

## Correlation-Based Clustering

## Cointegrated Pairs

- log P1 = c + beta * log P2 + error
- error_t = alpha * error_(t-1) + epsilon

Hypothesis Test
- H0: alpha = 1
- H1: alpha < 1 -> error is stationary -> P1 & P2 are cointegrated

In [76]:
def coint_for_pair(i, j, data, keys, conf):
    y = data.iloc[:, i]
    x = data.iloc[:, j]
    # coint test - returns (t_stat, pvalue, crit_vals)
    score, pvalue, _ = coint(y, x)
    return i, j, pvalue, (keys[i], keys[j]), (pvalue < conf)

def find_cointegrated_pairs_parallel(data, conf=0.05, n_jobs=-1):
    data = data.dropna()  # optional: ensures coint doesn't choke on NaNs
    n = data.shape[1]
    keys = list(data.columns)

    tuples = [(i, j) for i in range(n) for j in range(i+1, n)]

    results = Parallel(n_jobs=n_jobs, verbose=0)(
        delayed(coint_for_pair)(i, j, data, keys, conf)
        for i, j in tuples
    )

    beta_matrix = np.full((n, n), np.nan)
    pvalue_matrix = np.full((n, n), np.nan)
    # for i in range(n):
    #     beta_matrix[i,i] = 1
    #     pvalue_matrix[i,i] = 0
    pairs = []

    for i, j, pvalue, pair, is_coint in results:
        pvalue_matrix[i, j] = pvalue
        if not is_coint: 
            continue
        pairs.append(pair)
        # OLS
        y = data.iloc[:, i]
        x = data.iloc[:, j]
        x_mean = x.mean()
        y_mean = y.mean()
        dx = x - x_mean
        beta = (dx @ (y - y_mean)) / (dx @ dx)
        beta_matrix[i, j] = beta
            
    return beta_matrix, pvalue_matrix, pairs

In [77]:
subset = log_price_train.iloc[:, :50]
beta_matrix, pvalue_matrix, pairs = find_cointegrated_pairs_parallel(subset)

fig = go.Figure(data=go.Heatmap(
                   z=pvalue_matrix,
                   x=subset.columns, y=subset.columns,
                   hoverongaps = False,
                   zmin=0, zmax=1, colorscale="thermal_r",
))
fig.update_layout(width=800,height=800)
fig.show()

In [78]:
cols = list(subset.columns)
col_to_idx = {c:i for i,c in enumerate(cols)}

pairs_df = pd.DataFrame(pairs, columns=["sym1","sym2"])
pairs_df["pvalue"] = pairs_df.apply(
    lambda r: pvalue_matrix[col_to_idx[r["sym1"]], col_to_idx[r["sym2"]]],
    axis=1
)
pairs_df["beta"] = pairs_df.apply(
    lambda r: beta_matrix[col_to_idx[r["sym1"]], col_to_idx[r["sym2"]]],
    axis=1
)

pairs_df["name1"] = pairs_df["sym1"].map(sym2name)
pairs_df["name2"] = pairs_df["sym2"].map(sym2name)
pairs_df = pairs_df.sort_values("pvalue").reset_index(drop=True)
pairs_df.head(20)

,sym1,sym2,pvalue,beta,name1,name2
0,ACWI,CWB,0.000338,0.985111,iShares MSCI ACWI ETF,State Street SPDR Bloomberg Convertible Securi...
1,AOA,CZA,0.001195,0.712861,iShares Core 80/20 Aggressive Allocation ETF,Invesco Zacks Mid-Cap ETF
2,CZA,DDM,0.002196,0.662998,Invesco Zacks Mid-Cap ETF,ProShares Ultra Dow30
3,DBB,DRV,0.003945,0.172751,Invesco DB Base Metals Fund,Direxion Daily Real Estate Bear 3X Shares
4,ACWI,CEF,0.005014,-0.486818,iShares MSCI ACWI ETF,Sprott Physical Gold and Silver Trust Units
5,AIVL,DIA,0.005444,1.131021,WisdomTree U.S. AI Enhanced Value Fund,SPDR Dow Jones Industrial Average ETF
6,ACWI,DBP,0.005656,-0.634844,iShares MSCI ACWI ETF,Invesco DB Precious Metals Fund
7,AGQ,BIS,0.006300,0.612705,ProShares Ultra Silver,ProShares UltraShort Nasdaq Biotechnology
8,ACWI,AGQ,0.006810,-0.158367,iShares MSCI ACWI ETF,ProShares Ultra Silver
9,AIVL,DDM,0.007065,0.639727,WisdomTree U.S. AI Enhanced Value Fund,ProShares Ultra Dow30


In [92]:
fig = go.Figure()
df_ = price_df.iloc[:train_size]
pair = pairs_df.iloc[10, :2]
print("Correlations:", np.corrcoef(ret_df.iloc[:train_size][pair], rowvar=False)[0,1])
for sym in pair:
    fig.add_trace(go.Scatter(x=df_.index, y=df_[sym], name=sym))
fig.show()

Correlations: 0.9241308906063436


Problems:
- log_price cointegrated but returns not correlated - can fix after using clustering on returns correlation